# Centrality Measures with cuGraph and US Patent Citations.
This notebook will demonstrate RAPIDS cuGraph to do centrality calculations on US Patent data

# Downloading the data
Commenting out and running the below lines will download and expand the data into the directory the notebook expects it to be in.

In [ ]:
#!wget https://s3.amazonaws.com/data.patentsview.org/download/g_patent.tsv.zip
#!unzip ./_patent.tsv.zip
#!wget https://s3.amazonaws.com/data.patentsview.org/download/g_us_patent_citation.tsv.zip
#!unzip ./g_us_patent_citation.tsv.zip

We will create the dataframes using cudf and create the graphs with cuGraph.

In [ ]:
import cudf
import cugraph

This method takes a list of patents and goes out one hop. It returns the new list of all the patents which can
be used to in the same function to go out an additional hop.

In [ ]:
def next_hop(seeds):
    seed_df = seeds.to_frame('seed')
    source_df = citation_df.merge(seed_df, left_on='source', right_on='seed', how='inner')
    target_df = (citation_df.merge(seed_df, left_on='target', right_on='seed', how='inner'))
    links_df = cudf.concat([source_df,target_df])
    links_df = links_df.drop('seed', axis=1)
    new_seed_set = links_df.melt()
    new_seed_set = new_seed_set.drop('variable',axis=1)
    new_seed_set = new_seed_set['value'].drop_duplicates()
    return links_df.drop_duplicates(), new_seed_set

This function displays the top results of all the centrality algorithms

In [ ]:
# Print function
def print_centrality(k, dc, bc, kr, pr, ev):

    dc_top = dc.sort_values(by='degree_centrality', ascending=False).head(k).to_pandas()
    bc_top = bc.sort_values(by='betweenness_centrality', ascending=False).head(k).to_pandas()
    kr_top = kr.sort_values(by='katz_centrality', ascending=False).head(k).to_pandas()
    pr_top = pr.sort_values(by='pagerank', ascending=False).head(k).to_pandas()
    ev_top = ev.sort_values(by='eigenvector_centrality', ascending=False).head(k).to_pandas()
    
    df1_styler = dc_top.style.set_table_attributes("style='display:inline'").set_caption('Degree').hide(axis='index')
    df2_styler = bc_top.style.set_table_attributes("style='display:inline'").set_caption('Betweenness').hide(axis='index')
    df3_styler = kr_top.style.set_table_attributes("style='display:inline'").set_caption('Katz').hide(axis='index')
    df4_styler = pr_top.style.set_table_attributes("style='display:inline'").set_caption('PageRank').hide(axis='index')
    df5_styler = ev_top.style.set_table_attributes("style='display:inline'").set_caption('EigenVector').hide(axis='index')

    display_html(df1_styler._repr_html_()+df2_styler._repr_html_()+df3_styler._repr_html_()+df4_styler._repr_html_()+df5_styler._repr_html_(), raw=True)

After running the centrality algorithms, this function displays the supplied graph. The dataframe from any of the centrality calculations is also supplied along with the number of the highest ranking nodes we want to emphasize with labels and scale with size based on the centrality value.

In [ ]:
def draw_centrality_graph(graph_df, number_to_label, cent_df):
    import networkx as nx
    import matplotlib.pyplot as plt

    top_ones = cent_df.sort_values(by=cent_df.columns[0], ascending=False).head(number_to_label)[cent_df.columns[1]].to_pandas()
    print(top_ones)
    sizes = []
    nodes = []

    plt.rcParams['figure.figsize'] = [12, 8]
    nx_g = nx.from_pandas_edgelist(graph_df.to_pandas())

    for node in nx_g.nodes():
        cent_value =cent_df.loc[cent_df['vertex'] == node, cent_df.columns[0]].values[0]
        nodes.append(node)
        sizes.append((cent_value[0]*3000+10).item())

    pos = nx.spring_layout(nx_g)
    labels = {node: node for node in top_ones}
    nx.draw(nx_g, pos=pos, nodelist=nodes,font_color='red' ,node_size=sizes,node_color='grey', node_shape='o', linewidths=2, labels=labels)

This function executes the five Centrality Algorithms.

In [ ]:
# Compute Centrality
# the centrality calls are very straightforward with the graph being the primary argument
# we are using the default argument values for all centrality functions except where noted.

def compute_centrality(_graph, highest_degree=2000) :

    # Compute Degree Centrality
    _d = cugraph.degree_centrality(_graph)
    print("Degree Centrality done")

    # Compute the Betweenness Centrality
    # The k value is needed in large graphs as by default it will calculate the for every pair in the graph.
    # in this case we limit it to 100 samples at most.
    _b = cugraph.betweenness_centrality(_graph,k=100)
    print("Between Centrality done")

    #Compute Katz Centrality
    # alpha is set to one divided by the maximum degree in the graph. This will enable convergence
    # max_iter will determine how many iterations will be run before giving up if convergence doesn't occur
    # tol is the tolerance which determines the maximum difference that indicatates convergence 
    _k = cugraph.katz_centrality(_graph, max_iter=1000, tol=1.0e-2,alpha=1/highest_degree)
    print("Katz Centrality done")

    # Compute PageRank Centrality
    _p = cugraph.pagerank(_graph)
    print("Pagerank done")

    # Compute EigenVector Centrality
    _e = cugraph.eigenvector_centrality(_graph, max_iter=1000, tol=1.0e-3)
    print("Eigenvector Centrality done")

    return _d, _b, _k, _p, _e

Here we load the entire US patent citation edge list in to a cuDF dataframe.

In [ ]:
# load the citation graph
import cudf
citation_df = cudf.read_csv("../data/g_us_patent_citation.tsv",
                sep='\t',
                header=0,
                usecols=[0,2],
                names=["source", "target"],
                dtype={"source":str,"target":str},
)

In [ ]:
# Display the first five edges in the dataframe.
citation_df.iloc[:5]

The resulting dataframe has over 142 million edges

In [ ]:
len(citation_df)

Choose a patent of interest. In this case we will choose a patent representing an advance in visualizing machine learning models. Adding additional patents to the seed list can be done here.

In [ ]:
poi = ["10810491"]

Go out one hop from the patent(s) supplied.

In [ ]:
seed_series=cudf.Series(poi)
first_hop_df, first_set = next_hop(seed_series)

In [ ]:
len(first_hop_df)

In [ ]:
second_hop_df, second_hop_seeds = next_hop(first_set)
third_hop_df, third_hop_seeds = next_hop(second_hop_seeds)
fourth_hop_df, fourth_hop_seeds = next_hop(third_hop_seeds)


These are node (patent) counts at each hop.

In [ ]:
len(first_set), len(second_hop_seeds),len(third_hop_seeds),len(fourth_hop_seeds)

These are edge counts at each hop.

In [ ]:
len(first_hop_df), len(second_hop_df),len(third_hop_df),len(fourth_hop_df)

The contents of the dataframe at 2 hops

In [ ]:
second_hop_df

In [ ]:
%%time
G = cugraph.from_cudf_edgelist(second_hop_df,create_using=cugraph.Graph(directed=True),source='source', destination='target')

In [ ]:
%%time
dc, bc, kc, pr, ev = compute_centrality(G)

In [ ]:
from IPython.display import display_html

print_centrality(10, dc, bc, kc, pr, ev)

Calls the function that draws the graph with the specified number of the most central nodes labeled

In [ ]:
draw_centrality_graph(second_hop_df,12, pr)

Now load the enrichment data. We are only loading the patent_id and title but other columns are available.


In [ ]:
title_df = cudf.read_csv("/home/dacosta/data/g_patent.tsv",
                sep='\t',
                header=0,
                usecols=[0,3],
                names=["patent_id", "patent_title"],
                dtype={"patent_id":"str","patent_title":str},
)
len(title_df)


Lets run edge betweenness centrality to find the central edges in the graph.

In [ ]:
G_2_hops = cugraph.from_cudf_edgelist(second_hop_df,create_using=cugraph.Graph(directed=True),source='source', destination='target')
results=cugraph.edge_betweenness_centrality(G_2_hops).sort_values(ascending=False,by=['betweenness_centrality'])
results.head(10)

Merge the enrichment data with the highest ranking patent ids found above.

In [ ]:
# top number of Patents we are interested in
k=20

dc_top = dc.sort_values(by='degree_centrality', ascending=False).head(k)
bc_top = bc.sort_values(by='betweenness_centrality', ascending=False).head(k)
kr_top = kc.sort_values(by='katz_centrality', ascending=False).head(k)
pr_top = pr.sort_values(by='pagerank', ascending=False).head(k)
ev_top = ev.sort_values(by='eigenvector_centrality', ascending=False).head(k)

df_list = [dc_top, bc_top, kr_top, pr_top, ev_top]
combined = cudf.concat(df_list, axis=0)
combined = cudf.DataFrame(combined['vertex'])
print(combined.columns)
combined = combined.drop_duplicates()
enriched_df = title_df.merge(combined, left_on='patent_id', right_on='vertex', how='inner')
enriched_df = enriched_df.drop('vertex', axis=1 )

Display the results of the enriched dataframe. We do this in pandas in order to override the default max column width.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
enriched_df.to_pandas()

---
U.S. Patent and Trademark Office. “Data Download Tables.” PatentsView. Accessed [10/06/2024]. https://patentsview.org/ download/data-download-tables.

Data used is Licensed under Creative Commons 4.0 
https://creativecommons.org/licenses/by/4.0/

___
Copyright (c) 2024, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___